# Mapping with plotly express
plotly.express

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
import pandas as pd
import plotly.offline as pyo
import plotly.express as px

from datetime import datetime, timedelta
offset_hrs=-11

def get_utc_string(from_tz):
    from_tz_utc = datetime.strptime(from_tz, "%Y-%m-%d %H:%M") + timedelta(hours=offset_hrs) 
    ret_utc = from_tz_utc.strftime("%Y-%m-%dT%H:%M:%S")
    return ret_utc

def get_local_string(from_utc):
    from_tz_utc = datetime.strptime(from_utc[:-4], "%Y-%m-%dT%H:%M:%S.%f") - timedelta(hours=offset_hrs)
    ret_local = from_tz_utc.strftime("%Y/%m/%d %H:%M")
    return ret_local

time_from=get_utc_string('2021-01-01 14:00')
time_to=get_utc_string('2021-01-01 16:10')

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('saubury-001')

fe = Key('time').between(time_from, time_to);
response = table.scan(FilterExpression=fe)


items = response['Items']
lat_array = []
lon_array = []
vistime_array = []

for item in items:
    lat_array.append(item['Latitude'])
    lon_array.append(item['Longitude'])
    vistime_array.append(get_local_string(item['time']))


mydict = {'lat': lat_array, 'lon': lon_array, 'time' : vistime_array}
mydf = pd.DataFrame(data=mydict)
# print(mydf)


fig = px.line_mapbox(df, lat="lat", lon="lon", hover_name="time", zoom=3, height=300)
fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=12, mapbox_center_lat = -33.8,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()